<a href="https://colab.research.google.com/github/shahd1995913/Identifying-Machine-Generated-Text-/blob/main/Part1_multilingual_framework_for_detecting_biases_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================ Bias Analyzer

# ---- Auto install dependencies (safe & idempotent) ----
import sys, subprocess, importlib

def ensure(pkg, import_name=None):
    import_name = import_name or pkg
    try:
        importlib.import_module(import_name)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", pkg])
        importlib.invalidate_caches()

# Core libs
ensure("torch")
ensure("transformers")
ensure("numpy")
ensure("scipy")
ensure("nltk")
# Detoxify for toxicity (has CPU-only fallback if no GPU)
ensure("detoxify")

import re
import math
import numpy as np
from statistics import mean, pstdev
from dataclasses import dataclass
from typing import List, Dict, Any

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from detoxify import Detoxify
import nltk

# NLTK lightweight setup (we avoid punkt to keep footprint small)
# For sentiment we need vader_lexicon only
try:
    nltk.data.find("sentiment/vader_lexicon.zip")
except LookupError:
    nltk.download("vader_lexicon", quiet=True)

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# ---------------- Utilities ----------------

def split_sentences(text: str) -> List[str]:
    # Lightweight sentence splitter to avoid large NLTK punkt
    # Splits on .?! while preserving reasonable tokens
    raw = re.split(r'(?<=[\.\?\!])\s+', text.strip())
    # Clean and filter
    return [s.strip() for s in raw if s.strip()]

def tokenize_words(text: str) -> List[str]:
    return re.findall(r"[A-Za-zÀ-ÖØ-öø-ÿ0-9']+", text.lower())

def type_token_ratio(tokens: List[str]) -> float:
    if not tokens:
        return 0.0
    return len(set(tokens)) / max(1, len(tokens))

def repetition_ratio(tokens: List[str], n: int = 3) -> float:
    """
    Measures n-gram repetition (0..1). Higher -> more repetition (AI often repeats).
    """
    if len(tokens) < n:
        return 0.0
    ngrams = [" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    if not ngrams:
        return 0.0
    unique = len(set(ngrams))
    return 1.0 - (unique / len(ngrams))

def sentence_length_stats(sentences: List[str]) -> Dict[str, float]:
    lengths = [len(tokenize_words(s)) for s in sentences if s.strip()]
    if not lengths:
        return {"avg": 0.0, "std": 0.0, "cv": 0.0}
    avg = mean(lengths)
    std = pstdev(lengths) if len(lengths) > 1 else 0.0
    cv = (std / avg) if avg > 0 else 0.0
    return {"avg": float(avg), "std": float(std), "cv": float(cv)}

def softclip(x, lo, hi):
    return max(lo, min(hi, x))

# ---------------- Perplexity via GPT-2 ----------------
# We use GPT-2 to compute perplexity (lower perplexity -> more "model-like")
# Robustly handles long texts by sliding window.
class PerplexityScorer:
    def __init__(self, model_name: str = "gpt2", max_length: int = 1024, stride: int = 512, device: str = None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = device
        self.model.to(self.device)
        self.model.eval()
        self.max_length = max_length
        self.stride = stride

    @torch.no_grad()
    def perplexity(self, text: str) -> float:
        if not text.strip():
            return float("inf")
        encodings = self.tokenizer(text, return_tensors="pt")
        input_ids = encodings.input_ids.to(self.device)
        nlls = []
        for i in range(0, input_ids.size(1), self.stride):
            begin_loc = max(i + self.stride - self.max_length, 0)
            end_loc = min(i + self.stride, input_ids.size(1))
            trg_len = end_loc - i
            if trg_len <= 0:
                continue
            input_ids_slice = input_ids[:, begin_loc:end_loc]
            target_ids = input_ids_slice.clone()
            target_ids[:, :-trg_len] = -100  # mask everything except target region
            outputs = self.model(input_ids_slice, labels=target_ids)
            nll = outputs.loss * trg_len
            nlls.append(nll)
        if not nlls:
            return float("inf")
        ppl = torch.exp(torch.stack(nlls).sum() / max(1, input_ids.size(1))).item()
        # Guard against numeric issues
        if math.isinf(ppl) or math.isnan(ppl):
            return 1e6
        return float(ppl)

# ---------------- Toxicity & Sentiment ----------------

class RobustDetoxify:
    """
    Wrap Detoxify with CPU fallback and graceful degradation.
    """
    def __init__(self):
        # Use 'original' weights (multi-label) suitable for general toxicity
        try:
            self.model = Detoxify('original')
            self.ok = True
        except Exception:
            self.model = None
            self.ok = False

    def score(self, text: str) -> Dict[str, float]:
        if not self.ok or not text.strip():
            return {"toxicity": 0.0, "severe_toxicity": 0.0, "obscene": 0.0,
                    "threat": 0.0, "insult": 0.0, "identity_attack": 0.0}
        try:
            out = self.model.predict(text)
            # Ensure presence of keys
            for k in ["toxicity","severe_toxicity","obscene","threat","insult","identity_attack"]:
                if k not in out:
                    out[k] = 0.0
            return {k: float(out[k]) for k in out}
        except Exception:
            return {"toxicity": 0.0, "severe_toxicity": 0.0, "obscene": 0.0,
                    "threat": 0.0, "insult": 0.0, "identity_attack": 0.0}

# ---------------- Bias Lexicon ----------------

PROTECTED_TERMS = set([
    # Gender & identity
    "woman","women","man","men","female","male","nonbinary","trans","transgender","lgbt","gay","lesbian","bisexual","queer",
    # Race/ethnicity (examples; non-exhaustive, lowercase)
    "black","white","asian","latino","hispanic","arab","indian","african","european","indigenous","native","jewish","jews","muslim","christian","christians",
    # Religion (more)
    "islam","christianity","judaism","buddhist","hindu","sikh",
    # Age/disability
    "disabled","disability","handicapped","elderly","old","young","youth","senior",
    # Nationality (examples)
    "american","british","irish","french","german","italian","spanish","russian","ukrainian","chinese","japanese","korean","turkish","saudi","emirati","egyptian","jordanian","moroccan"
])

NEGATION_TERMS = set(["not","never","no","none","hardly","barely","without","less"])

def contains_protected_terms(text: str) -> bool:
    tokens = tokenize_words(text)
    return any(t in PROTECTED_TERMS for t in tokens)

# ---------------- Feature Engineering ----------------

@dataclass
class TextFeatures:
    perplexity: float
    ttr: float
    repetition: float
    sent_cv: float
    avg_sent_len: float

def extract_features(text: str, ppl_scorer: PerplexityScorer) -> TextFeatures:
    sentences = split_sentences(text)
    tokens = tokenize_words(text)
    stats = sentence_length_stats(sentences)
    ppl = ppl_scorer.perplexity(text)
    return TextFeatures(
        perplexity=ppl,
        ttr=type_token_ratio(tokens),
        repetition=repetition_ratio(tokens, n=3),
        sent_cv=stats["cv"],
        avg_sent_len=stats["avg"]
    )

# ---------------- AI Probability (Heuristic Model) ----------------
# We map features to a probability via a simple, well-behaved logistic model.
# Intuition:
#   • Lower perplexity  -> more AI-like
#   • Lower burstiness (CV) -> more AI-like
#   • Higher repetition -> more AI-like
#   • Lower TTR         -> more AI-like

def sigmoid(x): return 1 / (1 + math.exp(-x))

def ai_probability(feat: TextFeatures) -> float:
    # Normalize features to stable ranges
    # Perplexity is unbounded & skewed; clip to [5, 200] then invert.
    ppl = softclip(feat.perplexity, 5, 200)
    ppl_norm = (ppl - 5) / (200 - 5)  # 0 (very low ppl) -> 1 (high ppl)
    # We want lower perplexity => higher AI-ness, so invert:
    ppl_ai = 1.0 - ppl_norm

    # CV (0..2 typical); cap at 1.5
    cv = softclip(feat.sent_cv, 0.0, 1.5) / 1.5
    cv_ai = 1.0 - cv  # lower CV -> more AI-like

    # Repetition already 0..1 (higher -> AI-like)
    rep_ai = softclip(feat.repetition, 0.0, 1.0)

    # TTR (0..1) (lower -> AI-like)
    ttr_ai = 1.0 - softclip(feat.ttr, 0.0, 1.0)

    # Weighted sum -> logistic
    # Weights tuned for balanced influence
    z = (2.2 * ppl_ai) + (1.6 * cv_ai) + (1.2 * rep_ai) + (1.0 * ttr_ai) - 2.0
    return float(softclip(sigmoid(z), 0.01, 0.99))

# ---------------- Mixed Authorship Detection ----------------

def segment_ai_mix(text: str, ppl_scorer: PerplexityScorer) -> Dict[str, Any]:
    sentences = split_sentences(text)
    if not sentences:
        return {"ai_like": 0.0, "human_like": 0.0, "label": "insufficient_text"}
    # Group into chunks of ~4 sentences for stable scoring
    chunks = []
    for i in range(0, len(sentences), 4):
        chunk = " ".join(sentences[i:i+4])
        if chunk.strip():
            chunks.append(chunk)
    probs = []
    for ch in chunks:
        f = extract_features(ch, ppl_scorer)
        probs.append(ai_probability(f))
    if not probs:
        return {"ai_like": 0.0, "human_like": 0.0, "label": "insufficient_text"}
    ai_frac = float(np.mean([p > 0.6 for p in probs]))
    human_frac = float(np.mean([p < 0.4 for p in probs]))
    if ai_frac >= 0.3 and human_frac >= 0.3:
        label = "mixed"
    elif ai_frac >= 0.5:
        label = "ai_generated_likely"
    elif human_frac >= 0.5:
        label = "human_written_likely"
    else:
        label = "unclear"
    return {"ai_like": ai_frac, "human_like": human_frac, "label": label, "chunk_probs": probs}

# ---------------- Bias Scoring ----------------

def bias_scores(text: str, tox_model: RobustDetoxify) -> Dict[str, Any]:
    # Toxicity (0..1 for each)
    tox = tox_model.score(text)
    toxicity = float(tox.get("toxicity", 0.0))
    identity_attack = float(tox.get("identity_attack", 0.0))
    insult = float(tox.get("insult", 0.0))
    severe = float(tox.get("severe_toxicity", 0.0))
    threat = float(tox.get("threat", 0.0))

    # Sentiment extremity (abs compound)
    sent = sia.polarity_scores(text)
    sentiment_extremity = abs(float(sent.get("compound", 0.0)))  # 0..1

    # Mentions of protected groups
    has_protected = contains_protected_terms(text)

    # Bias aggregator:
    # Base toxicity weight
    base = 0.5 * toxicity + 0.1 * severe + 0.1 * threat + 0.15 * insult + 0.15 * identity_attack
    # If protected groups mentioned, amplify by sentiment extremity (negative contexts tend to be higher risk)
    protected_amp = (0.4 * sentiment_extremity) if has_protected else (0.15 * sentiment_extremity)
    score = softclip(base + protected_amp, 0.0, 1.0)

    # Map to 0..100
    overall = round(100.0 * score, 2)

    return {
        "overall_bias_score_0_100": overall,
        "toxicity": round(100 * toxicity, 2),
        "identity_attack": round(100 * identity_attack, 2),
        "insult": round(100 * insult, 2),
        "severe_toxicity": round(100 * severe, 2),
        "threat": round(100 * threat, 2),
        "sentiment_extremity_0_100": round(100 * sentiment_extremity, 2),
        "mentions_protected_groups": bool(has_protected)
    }

# ---------------- Main API ----------------

def analyze_text(text: str) -> Dict[str, Any]:
    text = (text or "").strip()
    if not text:
        return {"error": "Empty text. Please provide non-empty input."}

    # Init scorers
    ppl_scorer = PerplexityScorer(model_name="gpt2", max_length=1024, stride=512)
    tox_model = RobustDetoxify()

    # Global features & AI probability
    features = extract_features(text, ppl_scorer)
    ai_prob = ai_probability(features)
    mix = segment_ai_mix(text, ppl_scorer)

    # Labeling
    if mix["label"] == "mixed":
        label = "mixed_authorship_suspected"
    else:
        if ai_prob >= 0.65:
            label = "ai_generated_likely"
        elif ai_prob <= 0.35:
            label = "human_written_likely"
        else:
            label = "unclear"

    # Bias scoring
    bias = bias_scores(text, tox_model)

    # Package results
    return {
        "ai_detection": {
            "ai_probability_0_1": round(ai_prob, 4),
            "label": label,
            "global_features": {
                "perplexity": round(features.perplexity, 2),
                "type_token_ratio": round(features.ttr, 4),
                "repetition_3gram": round(features.repetition, 4),
                "sentence_length_cv": round(features.sent_cv, 4),
                "avg_sentence_length_tokens": round(features.avg_sent_len, 2),
            },
            "segment_mix_analysis": {
                "ai_like_fraction": round(float(mix.get("ai_like", 0.0)), 3),
                "human_like_fraction": round(float(mix.get("human_like", 0.0)), 3),
                "segment_label": mix.get("label", "n/a"),
                "segment_ai_probs": [round(float(p), 3) for p in mix.get("chunk_probs", [])]
            }
        },
        "bias_assessment": bias
    }

# ---------------- Example Usage ----------------
if __name__ == "__main__":
    # You can replace this sample with any text, or use input() for quick testing.
    sample_text = (
        "Artificial intelligence models have transformed how we summarize and write content. "
        "This paragraph is intentionally neutral. However, claims that any specific nationality is inherently better at science "
        "are misguided and not supported by evidence."
    )

    print("=== AI & Bias Analyzer ===")
    print("Provide your text below. Press Enter to use the built-in sample.\n")
    try:
        user_inp = input("Paste text (or press Enter to analyze the sample): ").strip()
    except EOFError:
        user_inp = ""

    text = user_inp if user_inp else sample_text
    results = analyze_text(text)
    import json
    print(json.dumps(results, indent=2, ensure_ascii=False))


KeyboardInterrupt: 

In [ ]:
# -*- coding: utf-8 -*-
"""
AI-generated Content & Bias Detection System
--------------------------------------------
This system detects:
1. Whether text is written by a Human or AI (using Hugging Face zero-shot models).
2. Estimates degree of bias in the text:
   - Linguistic bias (negative/exclusionary words).
   - Religious or cultural bias.
   - Lack of diversity in sources/terms.

Author: Shahd
"""

from transformers import pipeline
import re

# ------------------------------
# Load Hugging Face Pipelines
# ------------------------------
# For AI/Human classification (zero-shot approach)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Sentiment analysis to help detect negative connotations
sentiment_analyzer = pipeline("sentiment-analysis")

# ------------------------------
# Bias Dictionaries (demo version)
# ------------------------------
bias_keywords = {
    "linguistic": ["غبي", "جاهل", "متخلف", "إرهابي", "عنصري", "حقير"],
    "religious": {
        "مسيحي": ["كافر", "ضال"],
        "شيعي": ["رافضي", "منحرف"],
        "سني": ["ناصبي", "متشدد"],
        "يهودي": ["ملعون", "مؤامرة"]
    },
    "cultural": ["بدائي", "غير متحضر", "أقل شأنا", "دوني"]
}

# ------------------------------
# Functions
# ------------------------------
def detect_ai_generated(text):
    """
    Classify text as Human-written or AI-generated using zero-shot classification.
    """
    labels = ["Human-written", "AI-generated"]
    result = classifier(text, labels)
    return result['labels'][0], result['scores'][0]

def detect_bias(text):
    """
    Detect bias in text:
    - Linguistic
    - Religious / Cultural
    - Diversity of terms
    """
    findings = []
    bias_score = 0

    # Check linguistic bias
    for word in bias_keywords["linguistic"]:
        if word in text:
            findings.append(f"انحياز لغوي: يحتوي على كلمة '{word}'")
            bias_score += 15

    # Check religious/cultural bias
    for group, negatives in bias_keywords["religious"].items():
        for neg in negatives:
            if neg in text:
                findings.append(f"انحياز ديني/ثقافي ضد '{group}': باستخدام كلمة '{neg}'")
                bias_score += 20

    for word in bias_keywords["cultural"]:
        if word in text:
            findings.append(f"انحياز ثقافي: يحتوي على وصف '{word}'")
            bias_score += 10

    # Sentiment check
    sentiment = sentiment_analyzer(text[:512])[0]
    if sentiment['label'] == "NEGATIVE":
        findings.append("النص يحتوي على نبرة سلبية عامة")
        bias_score += 10

    # Normalize bias score (0–100)
    bias_score = min(bias_score, 100)

    # Output result
    if not findings:
        findings.append("لا توجد مؤشرات واضحة على التحيز")

    return bias_score, findings

# ------------------------------
# Demo
# ------------------------------
if __name__ == "__main__":
    sample_text = """
    التاريخ الإسلامي يوضح أن المذهب الشيعي كان دائما منحرفا عن الحق،
    والسنة فقط هم أصحاب الحقيقة بينما الآخرون ضالون.
    """

    # Detect AI vs Human
    label, score = detect_ai_generated(sample_text)
    print("🔍 تصنيف النص:")
    print(f"- {label} (درجة الثقة: {score:.2f})\n")

    # Detect Bias
    bias_score, findings = detect_bias(sample_text)
    print("⚖️ تحليل الانحياز:")
    print(f"- درجة الانحياز: {bias_score}%")
    for f in findings:
        print(f"- {f}")

    # Example of summary
    if bias_score > 0:
        print(f"\n📌 النتيجة المبدئية: النص يحتوي على انحياز بنسبة {bias_score}% ضد المذهب الشيعي في عرضه للتاريخ الإسلامي.")
    else:
        print("\n📌 النتيجة المبدئية: النص لا يحتوي على انحياز واضح.")


In [ ]:
# AI-Generated Content and Bias Detection System with Harm Level Classification
# Author: Shahd's Assistant
# -------------------------------------------------------------

from transformers import pipeline
import re

# -------------------------------------------------------------
# 1. Load Models
# -------------------------------------------------------------
# Zero-shot classification for AI vs Human content
ai_detector = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Sentiment analysis for detecting negative/biased tone
sentiment_model = pipeline("sentiment-analysis")

# Zero-shot classification for harm level
harm_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# -------------------------------------------------------------
# 2. Define Bias Detection Function
# -------------------------------------------------------------
def detect_bias(text):
    biases = {
        "Linguistic Bias": ["negative", "exclusionary", "offensive", "derogatory"],
        "Religious/Cultural Bias": ["christianity", "islam", "judaism", "shiite", "sunni", "hindu", "buddhism"],
        "Lack of Diversity": ["western perspective", "single source", "no diversity"]
    }

    detected_biases = []
    for bias_type, keywords in biases.items():
        for word in keywords:
            if re.search(r"\b" + re.escape(word) + r"\b", text.lower()):
                detected_biases.append((bias_type, word))

    return detected_biases

# -------------------------------------------------------------
# 3. Harm Level Classification
# -------------------------------------------------------------
def classify_harm_level(text):
    harm_labels = [
        "Misinformation",
        "Cultural Exclusion",
        "Religious Bias",
        "Hate Speech",
        "Sensitive Content",
        "Neutral / Safe"
    ]
    harm_result = harm_classifier(text, harm_labels)
    return harm_result

# -------------------------------------------------------------
# 4. Main Analysis Function
# -------------------------------------------------------------
def analyze_text(text):
    # Step 1: Detect AI vs Human
    ai_result = ai_detector(text, candidate_labels=["AI-generated", "Human-written"])

    # Step 2: Sentiment analysis
    sentiment_result = sentiment_model(text)

    # Step 3: Bias detection
    detected_biases = detect_bias(text)

    # Step 4: Harm level classification
    harm_result = classify_harm_level(text)

    # ---------------------------------------------------------
    # Print results clearly in English
    # ---------------------------------------------------------
    print("\n=== AI Content Detection ===")
    print(f"Prediction: {ai_result['labels'][0]} with confidence {ai_result['scores'][0]:.2f}")

    print("\n=== Sentiment Analysis ===")
    print(f"Sentiment: {sentiment_result[0]['label']} (score: {sentiment_result[0]['score']:.2f})")

    print("\n=== Bias Detection ===")
    if detected_biases:
        for bias in detected_biases:
            print(f"- Potential {bias[0]} detected (keyword: {bias[1]})")
    else:
        print("No explicit linguistic/religious/cultural bias detected.")

    print("\n=== Harm Level Classification ===")
    for label, score in zip(harm_result['labels'], harm_result['scores']):
        print(f"- {label}: {score:.2f}")

    print("\n=== Final Example Output ===")
    if detected_biases:
        bias_example = f"The text contains approximately {round(harm_result['scores'][0]*100, 1)}% indication of {harm_result['labels'][0]}."
        print(bias_example)
    else:
        print("The text seems mostly neutral with minimal bias signals.")

# -------------------------------------------------------------
# 5. Run Example
# -------------------------------------------------------------
sample_text = """
This article claims that one religious group is always wrong while ignoring other historical perspectives.
It presents only a western perspective and excludes cultural diversity.
"""

analyze_text(sample_text)


In [ ]:
from transformers import pipeline
from textblob import TextBlob
import re

# ------------------------------------------------
# Load Models
# ------------------------------------------------
# Zero-shot classifier to detect AI vs Human written text
ai_detector = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Sentiment analysis for linguistic bias
sentiment_analyzer = pipeline("sentiment-analysis")

# ------------------------------------------------
# Helper Functions
# ------------------------------------------------

def detect_ai_generated(text):
    labels = ["AI-generated", "Human-written"]
    result = ai_detector(text, labels)
    return result

def analyze_bias(text):
    sentiment = sentiment_analyzer(text)[0]
    sentiment_label = sentiment['label']
    sentiment_score = sentiment['score']

    # Simple keyword-based bias check (Arabic terms)
    religious_bias = re.findall(r"(شيعة|سنة|مسيحي|يهودي)", text)
    cultural_bias = re.findall(r"(غرب|شرق|عربي|أوروبي)", text)

    bias_report = []
    if religious_bias:
        bias_report.append(f"Religious/Cultural bias detected: {religious_bias}")
    if cultural_bias:
        bias_report.append(f"Cultural/Regional bias detected: {cultural_bias}")

    return {
        "sentiment": sentiment_label,
        "sentiment_score": round(sentiment_score, 3),
        "bias_details": bias_report if bias_report else ["No explicit bias detected."]
    }

def harm_level_classification(text):
    labels = [
        "Misinformation",
        "Cultural Exclusion",
        "Sensitive Content",
        "Neutral/Harmless"
    ]
    result = ai_detector(text, labels)
    return result

# ------------------------------------------------
# Main Analysis
# ------------------------------------------------
def analyze_text(text):
    ai_result = detect_ai_generated(text)
    bias_result = analyze_bias(text)
    harm_result = harm_level_classification(text)

    print("=== Text Analysis Report ===")
    print(f"Input Text: {text}")

    print("\n[AI Detection]")
    print(f"Prediction: {ai_result['labels'][0]} (Confidence: {round(ai_result['scores'][0], 3)})")

    print("\n[Linguistic and Cultural Bias Detection]")
    print(f"Sentiment: {bias_result['sentiment']} (Score: {bias_result['sentiment_score']})")
    for detail in bias_result['bias_details']:
        print(f"- {detail}")

    print("\n[Harm Level Classification]")
    print(f"Most likely category: {harm_result['labels'][0]} (Confidence: {round(harm_result['scores'][0], 3)})")

    # ------------------------------------------------
    # Final Example Output (automatic summary)
    # ------------------------------------------------
    print("\n=== Final Example Output ===")
    if bias_result['bias_details'] and "No explicit bias detected." not in bias_result['bias_details']:
        bias_example = f"The text contains approximately {round(harm_result['scores'][0]*100, 1)}% indication of {harm_result['labels'][0]}."
        print(bias_example)
    else:
        print("The text seems mostly neutral with minimal bias signals.")
    print("=============================\n")

# ------------------------------------------------
# Test with Arabic Input (Results in English)
# ------------------------------------------------
test_text = "التاريخ الإسلامي يركز فقط على إنجازات السنة دون ذكر مساهمات الشيعة."
analyze_text(test_text)
